In [1]:
#Import general lib

import pandas as pd
import numpy as np

In [2]:
#My jupter-notebbok was having some problem import xgboost from my system , hence i had to use 

import sys
print(sys.base_prefix)

import pip
pip.main(['install', 'xgboost'])

c:\users\shelvi garg\appdata\local\programs\python\python38


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the 'c:\users\shelvi garg\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


0

In [3]:
import xgboost

In [4]:
#Reading training data
data = pd.read_csv(r"C:\Users\Shelvi Garg\Desktop\Code\data-science\JOBATHON 1\TRAIN.csv")

In [5]:
data.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0


In [6]:
#exploring column names
data.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Response'],
      dtype='object')

In [7]:
data.shape

(50882, 14)

In [8]:
#processing the data: replacing NaN values with most frequent values in respective columns

for i in range(14):
     (data.iloc[:,i]).fillna(data.iloc[:,i].mode().iloc[0],inplace=True)

print(data)

          ID City_Code  Region_Code Accomodation_Type Reco_Insurance_Type  \
0          1        C3         3213            Rented          Individual   
1          2        C5         1117             Owned               Joint   
2          3        C5         3732             Owned          Individual   
3          4       C24         4378             Owned               Joint   
4          5        C8         2190            Rented          Individual   
...      ...       ...          ...               ...                 ...   
50877  50878        C4          845            Rented          Individual   
50878  50879        C5         4188            Rented          Individual   
50879  50880        C1          442            Rented          Individual   
50880  50881        C1            4             Owned               Joint   
50881  50882        C3         3866            Rented          Individual   

       Upper_Age  Lower_Age Is_Spouse Health Indicator  \
0             36 

In [9]:
data

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,1.0,3.0,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,X1,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50877,50878,C4,845,Rented,Individual,22,22,No,X3,1.0,3.0,18,7704.0,0
50878,50879,C5,4188,Rented,Individual,27,27,No,X3,7.0,3.0,4,5408.0,0
50879,50880,C1,442,Rented,Individual,63,63,No,X2,14+,1.0,12,11374.0,0
50880,50881,C1,4,Owned,Joint,71,49,No,X2,2.0,2.0,16,28179.2,1


In [10]:
#description of our training data 

data.describe()

,ID,Region_Code,Upper_Age,Lower_Age,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
count,50882.000000,50882.000000,50882.000000,50882.000000,50882.000000,50882.000000,50882.000000,50882.000000
mean,25441.500000,1732.788707,44.856275,42.738866,2.662415,15.115188,14183.950069,0.239947
std,14688.512535,1424.081652,17.310271,17.319375,0.841994,6.340663,6590.074873,0.427055
min,1.000000,1.000000,18.000000,16.000000,1.000000,1.000000,2280.000000,0.000000
25%,12721.250000,523.000000,28.000000,27.000000,2.000000,12.000000,9248.000000,0.000000
50%,25441.500000,1391.000000,44.000000,40.000000,3.000000,17.000000,13178.000000,0.000000
75%,38161.750000,2667.000000,59.000000,57.000000,3.000000,20.000000,18096.000000,0.000000
max,50882.000000,6194.000000,75.000000,75.000000,4.000000,22.000000,43350.400000,1.000000


In [11]:
#assigning target
y = data.Response

In [12]:
#pre-processing the 2 columns which are combination of str+int,as one-hot decoder cant be applied to these

print(data['Health Indicator'].value_counts())

data["Holding_Policy_Duration"].value_counts()

X1    24701
X2    10332
X3     6762
X4     5743
X5     1727
X6     1280
X7      196
X8       78
X9       63
Name: Health Indicator, dtype: int64


1.0     24750
14+      4335
2.0      4260
3.0      3586
4.0      2771
5.0      2362
6.0      1894
7.0      1645
8.0      1316
9.0      1114
10.0      813
11.0      546
12.0      513
13.0      511
14.0      466
Name: Holding_Policy_Duration, dtype: int64

In [13]:
#pre-processing the 2 columns which are combination of str+int,as one-hot decoder cant be applied to these

data['Health Indicator'].replace({"X1":1,"X2":2,"X3":3,"X4":4,"X5":5,"X6":6,"X7":7,"X8":8,"X9":9},inplace=True)

data['Holding_Policy_Duration'].replace({"14+":"15"},inplace=True)
#need to replace string with float
data['Holding_Policy_Duration'].replace({"15":int("15")},inplace=True)

In [14]:
data = data.reset_index()

In [15]:
data

,index,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,0,1,C3,3213,Rented,Individual,36,36,No,1,15,3.0,22,11628.0,0
1,1,2,C5,1117,Owned,Joint,75,22,No,2,1.0,3.0,22,30510.0,0
2,2,3,C5,3732,Owned,Individual,32,32,No,1,1.0,1.0,19,7450.0,1
3,3,4,C24,4378,Owned,Joint,52,48,No,1,15,3.0,19,17780.0,0
4,4,5,C8,2190,Rented,Individual,44,44,No,2,3.0,1.0,16,10404.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50877,50877,50878,C4,845,Rented,Individual,22,22,No,3,1.0,3.0,18,7704.0,0
50878,50878,50879,C5,4188,Rented,Individual,27,27,No,3,7.0,3.0,4,5408.0,0
50879,50879,50880,C1,442,Rented,Individual,63,63,No,2,15,1.0,12,11374.0,0
50880,50880,50881,C1,4,Owned,Joint,71,49,No,2,2.0,2.0,16,28179.2,1


#One hot encoding

In [16]:
# Import label encoder 

from sklearn import preprocessing
# label_encoder object knows how to understand word labels. 

label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'. 

data['City_Code']= label_encoder.fit_transform(data['City_Code'])
data['Accomodation_Type']= label_encoder.fit_transform(data['Accomodation_Type'])
data['Reco_Insurance_Type']= label_encoder.fit_transform(data['Reco_Insurance_Type'])
data['Is_Spouse']= label_encoder.fit_transform(data['Is_Spouse'])


print(data.dtypes)

#Need to convert the object type before feeding to xgboost
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
data['Holding_Policy_Duration'] = lbl.fit_transform(data['Holding_Policy_Duration'].astype(str))

print(data.dtypes)


print(data.head())


index                        int64
ID                           int64
City_Code                    int32
Region_Code                  int64
Accomodation_Type            int32
Reco_Insurance_Type          int32
Upper_Age                    int64
Lower_Age                    int64
Is_Spouse                    int32
Health Indicator             int64
Holding_Policy_Duration     object
Holding_Policy_Type        float64
Reco_Policy_Cat              int64
Reco_Policy_Premium        float64
Response                     int64
dtype: object
index                        int64
ID                           int64
City_Code                    int32
Region_Code                  int64
Accomodation_Type            int32
Reco_Insurance_Type          int32
Upper_Age                    int64
Lower_Age                    int64
Is_Spouse                    int32
Health Indicator             int64
Holding_Policy_Duration      int32
Holding_Policy_Type        float64
Reco_Policy_Cat              int64
Reco_P

In [17]:


#Defining Features AS x

x = data[["City_Code","Region_Code",
                 "Accomodation_Type","Reco_Insurance_Type","Upper_Age",
                 "Lower_Age","Is_Spouse","Health Indicator","Holding_Policy_Duration",
                 "Holding_Policy_Type","Reco_Policy_Cat","Reco_Policy_Premium"]]

In [18]:
#Building the model using XG BOOST

from xgboost import XGBRegressor

xg_model = XGBRegressor()
xg_model.fit(x,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [19]:
#making predictings on training data

prediction_train = xg_model.predict(x)
print(prediction_train)

[0.3508137  0.05995572 0.36832386 ... 0.26306158 0.31912315 0.19651406]


In [20]:
#calculating the mean error 

from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y, prediction_train))

0.30513393498644376


In [21]:
#loading testing data

test= pd.read_csv(r"C:\Users\Shelvi Garg\Desktop\Code\data-science\JOBATHON 1\TEST.csv")
test.shape

(21805, 13)

In [22]:
test.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0
1,50884,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8
2,50885,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0
3,50886,C3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0
4,50887,C1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0


In [23]:
#Processing Test data to replace NaN Values

for i in range(12):
     (test.iloc[:,i]).fillna(test.iloc[:,i].mode().iloc[0],inplace=True)

print(test)
test.shape

          ID City_Code  Region_Code Accomodation_Type Reco_Insurance_Type  \
0      50883        C1          156             Owned          Individual   
1      50884        C4            7             Owned               Joint   
2      50885        C1          564            Rented          Individual   
3      50886        C3         1177            Rented          Individual   
4      50887        C1          951             Owned          Individual   
...      ...       ...          ...               ...                 ...   
21800  72683        C3         1044             Owned          Individual   
21801  72684        C4          266             Owned          Individual   
21802  72685       C12         2470             Owned          Individual   
21803  72686       C10         1676            Rented          Individual   
21804  72687        C2         3617             Owned          Individual   

       Upper_Age  Lower_Age Is_Spouse Health Indicator  \
0             30 

(21805, 13)

In [24]:
#pre-processing the 2 columns which are combination of str+int,as one-hot decoder cant be applied to these
#same as for training data
test['Health Indicator'].replace({"X1":1,"X2":2,"X3":3,"X4":4,"X5":5,"X6":6,"X7":7,"X8":8,"X9":9},inplace=True)

test['Holding_Policy_Duration'].replace({"14+":"15"},inplace=True)
test['Holding_Policy_Duration'].replace({"15":int("15")},inplace=True)


In [31]:
#One-hot-encoder

# Import label encoder 

from sklearn import preprocessing
# label_encoder object knows how to understand word labels. 

label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'. 

test['City_Code']= label_encoder.fit_transform(test['City_Code'])
test['Accomodation_Type']= label_encoder.fit_transform(test['Accomodation_Type'])
test['Reco_Insurance_Type']= label_encoder.fit_transform(test['Reco_Insurance_Type'])
test['Is_Spouse']= label_encoder.fit_transform(test['Is_Spouse'])


#Need to convert the object type before feeding to xgboost
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
test['Holding_Policy_Duration'] = lbl.fit_transform(test['Holding_Policy_Duration'].astype(str))

print(test.dtypes)


ID                           int64
City_Code                    int64
Region_Code                  int64
Accomodation_Type            int64
Reco_Insurance_Type          int64
Upper_Age                    int64
Lower_Age                    int64
Is_Spouse                    int64
Health Indicator             int64
Holding_Policy_Duration      int32
Holding_Policy_Type        float64
Reco_Policy_Cat              int64
Reco_Policy_Premium        float64
dtype: object


In [32]:
test.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,0,156,0,0,30,30,0,1,11,3.0,5,11934.0
1,50884,30,7,0,1,69,68,1,1,8,3.0,18,32204.8
2,50885,0,564,1,0,28,28,0,3,7,4.0,17,9240.0
3,50886,22,1177,1,0,23,23,0,3,8,3.0,18,9086.0
4,50887,0,951,0,0,75,75,0,3,6,3.0,5,22534.0


In [33]:
test.shape

(21805, 13)

In [28]:
test.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium'],
      dtype='object')

In [34]:
#filtering columns we need to provide to predict

test_data= test[["City_Code","Region_Code",
                 "Accomodation_Type","Reco_Insurance_Type","Upper_Age",
                 "Lower_Age","Is_Spouse","Health Indicator","Holding_Policy_Duration",
                 "Holding_Policy_Type","Reco_Policy_Cat","Reco_Policy_Premium"]]


In [35]:
#predicting
test_prediction = xg_model.predict(test_data)


In [36]:
test_prediction

array([0.2943197 , 0.1995237 , 0.27321088, ..., 0.150541  , 0.26620662,
       0.07287499], dtype=float32)

In [37]:
#length of resultant array;

len(test_prediction)

21805

In [47]:
#converting result array to pandas dataframe and finally to CSV as to save for submission

pd.DataFrame(test_prediction).to_csv(r"C:\Users\Shelvi Garg\Desktop\Code\data-science\result3.csv")

In [48]:
result_data = pd.read_csv(r"C:\Users\Shelvi Garg\Desktop\Code\data-science\result3.csv")

In [49]:
print(result_data)

       Unnamed: 0         0
0               0  0.294320
1               1  0.199524
2               2  0.273211
3               3  0.211898
4               4  0.220838
...           ...       ...
21800       21800  0.149179
21801       21801  0.169361
21802       21802  0.150541
21803       21803  0.266207
21804       21804  0.072875

[21805 rows x 2 columns]


In [50]:
result_data.isnull().values.any()

False

In [43]:
result_data.shape

(21805, 2)